In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import gc

In [ ]:
FORECAST_HORIZON = 28
LOOKBACK_WINDOW = 56  # Use 56 days of history to predict 28 days
BATCH_SIZE = 128
EPOCHS = 50
LEARNING_RATE = 0.001
HIDDEN_SIZE = 128
NUM_BLOCKS = 3
AGGREGATION_LEVEL = 'store_category'  # Aggregate to reduce series count

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class NBeatsBlock(nn.Module):
    #Single N-BEATS block
    def __init__(self, input_size, output_size, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        
        # Backcast should match input size, forecast should match output size
        self.theta_b = nn.Linear(hidden_size, input_size)   # Backcast matches input
        self.theta_f = nn.Linear(hidden_size, output_size)  # Forecast matches output
        
    def forward(self, x):
        h = torch.relu(self.fc1(x))
        h = torch.relu(self.fc2(h))
        h = torch.relu(self.fc3(h))
        h = torch.relu(self.fc4(h))
        
        backcast = self.theta_b(h)  # Shape: (batch, input_size)
        forecast = self.theta_f(h)  # Shape: (batch, output_size)
        
        return backcast, forecast

In [ ]:
class NBeatsNet(nn.Module):
    #N-BEATS Network
    def __init__(self, input_size, output_size, hidden_size=128, num_blocks=3):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.blocks = nn.ModuleList([
            NBeatsBlock(input_size, output_size, hidden_size)
            for _ in range(num_blocks)
        ])
        
    def forward(self, x):
        residuals = x  # Start with input (shape: batch, input_size)
        forecast = torch.zeros(x.size(0), self.output_size, device=x.device)
        
        for block in self.blocks:
            block_backcast, block_forecast = block(residuals)
            residuals = residuals - block_backcast  # Both have shape (batch, input_size)
            forecast = forecast + block_forecast     # Both have shape (batch, output_size)
            
        return forecast

In [ ]:
class TimeSeriesDataset(Dataset):
    #Time series dataset for N-BEATS
    def __init__(self, data, lookback, horizon, train=True):
        self.data = data
        self.lookback = lookback
        self.horizon = horizon
        self.train = train
        
        # Create sequences
        self.X = []
        self.y = []
        
        for i in range(lookback, len(data) - horizon + 1):
            self.X.append(data[i - lookback:i])
            if train:
                self.y.append(data[i:i + horizon])
        
        self.X = np.array(self.X, dtype=np.float32)
        if train:
            self.y = np.array(self.y, dtype=np.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.train:
            return torch.FloatTensor(self.X[idx]), torch.FloatTensor(self.y[idx])
        else:
            return torch.FloatTensor(self.X[idx])

In [ ]:
sales = pd.read_csv('/kaggle/input/forecasting-data/sales_train_validation.csv')
calendar = pd.read_csv('/kaggle/input/forecasting-data/calendar.csv')

day_cols = [col for col in sales.columns if col.startswith('d_')]
print(f" Loaded {len(day_cols)} days of data")


In [ ]:
def create_aggregated_series(sales_df, level):
    #Aggregate sales to reduce number of series
    if level == 'store_category':
        group_cols = ['store_id', 'cat_id']
    elif level == 'store_dept':
        group_cols = ['store_id', 'dept_id']
    elif level == 'category':
        group_cols = ['cat_id']
    elif level == 'store':
        group_cols = ['store_id']
    else:
        raise ValueError(f"Unknown level: {level}")
    
    agg_sales = sales_df.groupby(group_cols)[day_cols].sum().reset_index()
    
    if len(group_cols) > 1:
        agg_sales['series_id'] = agg_sales[group_cols].apply(lambda x: '_'.join(x), axis=1)
    else:
        agg_sales['series_id'] = agg_sales[group_cols[0]]
    
    return agg_sales

agg_sales = create_aggregated_series(sales, AGGREGATION_LEVEL)
n_series = len(agg_sales)

In [ ]:
models = {}
series_results = []

for idx, row in agg_sales.iterrows():
    series_id = row['series_id']
    print(f"\n   [{idx+1}/{n_series}] Training {series_id}...")
    
    # Get time series data
    ts_data = np.array([row[col] for col in day_cols], dtype=np.float32)
    
    # Normalize data
    scaler = StandardScaler()
    ts_data_scaled = scaler.fit_transform(ts_data.reshape(-1, 1)).flatten()
    
    # Split train/validation
    train_data = ts_data_scaled[:-FORECAST_HORIZON]
    val_data = ts_data_scaled
    
    # Create datasets
    train_dataset = TimeSeriesDataset(train_data, LOOKBACK_WINDOW, FORECAST_HORIZON, train=True)
    
    if len(train_dataset) < BATCH_SIZE:
        print(f"      Warning: Not enough data for {series_id}, skipping...")
        continue
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # Initialize model
    model = NBeatsNet(
        input_size=LOOKBACK_WINDOW,
        output_size=FORECAST_HORIZON,
        hidden_size=HIDDEN_SIZE,
        num_blocks=NUM_BLOCKS
    ).to(device)
    
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Training loop
    model.train()
    best_loss = float('inf')
    patience = 5
    patience_counter = 0
    
    for epoch in range(EPOCHS):
        epoch_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        
        if (epoch + 1) % 10 == 0:
            print(f"      Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}")
        
        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"      Early stopping at epoch {epoch+1}")
                break
    
    # Validation prediction
    model.eval()
    with torch.no_grad():
        # Use last LOOKBACK_WINDOW days to predict
        last_sequence = torch.FloatTensor(train_data[-LOOKBACK_WINDOW:]).unsqueeze(0).to(device)
        prediction_scaled = model(last_sequence).cpu().numpy().flatten()
    
    # Denormalize
    prediction = scaler.inverse_transform(prediction_scaled.reshape(-1, 1)).flatten()
    prediction = np.maximum(prediction, 0)  # Ensure non-negative
    
    # Get actual values
    actual = ts_data[-FORECAST_HORIZON:]
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(actual, prediction))
    mae = mean_absolute_error(actual, prediction)
    
    print(f"      ✓ RMSE: {rmse:.2f}, MAE: {mae:.2f}")
    
    # Store results
    models[series_id] = {'model': model, 'scaler': scaler}
    series_results.append({
        'series_id': series_id,
        'rmse': rmse,
        'mae': mae,
        'predictions': prediction,
        'actuals': actual
    })
    
    # Clear memory
    del model, train_dataset, train_loader
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
series_results_df = pd.DataFrame([{
    'series_id': r['series_id'],
    'rmse': r['rmse'],
    'mae': r['mae']
} for r in series_results])

print("\nPer-Series Results:")
print(series_results_df.to_string(index=False))

# Combine all predictions
all_predictions = np.concatenate([r['predictions'] for r in series_results])
all_actuals = np.concatenate([r['actuals'] for r in series_results])

# Daily aggregate
daily_predictions = np.zeros(FORECAST_HORIZON)
daily_actuals = np.zeros(FORECAST_HORIZON)

for r in series_results:
    daily_predictions += r['predictions']
    daily_actuals += r['actuals']

# Calculate metrics
overall_rmse = np.sqrt(mean_squared_error(all_actuals, all_predictions))
overall_mae = mean_absolute_error(all_actuals, all_predictions)

daily_rmse = np.sqrt(mean_squared_error(daily_actuals, daily_predictions))
daily_mae = mean_absolute_error(daily_actuals, daily_predictions)

mask = all_actuals != 0
mape = np.mean(np.abs((all_actuals[mask] - all_predictions[mask]) / all_actuals[mask])) * 100

print(f"\n{'='*80}")
print("N-BEATS PERFORMANCE METRICS")
print(f"{'='*80}")

print(f"\n SERIES-LEVEL METRICS ({len(series_results)} series):")
print(f"   RMSE:    {overall_rmse:.2f}")
print(f"   MAE:     {overall_mae:.2f}")
print(f"   MAPE:    {mape:.2f}%")

print(f"\n DAILY AGGREGATE METRICS (28 days):")
print(f"   RMSE:    {daily_rmse:.2f}")
print(f"   MAE:     {daily_mae:.2f}")
print(f"   Avg Daily Sales (Actual):    {daily_actuals.mean():,.0f} units")
print(f"   Avg Daily Sales (Predicted): {daily_predictions.mean():,.0f} units")

# Comparisons
baseline_rmse = 6922
lightgbm_rmse = 5200

improvement_baseline = ((baseline_rmse - daily_rmse) / baseline_rmse) * 100
improvement_lgbm = ((lightgbm_rmse - daily_rmse) / lightgbm_rmse) * 100

print(f"\n COMPARISON:")
print(f"   vs Baseline (MA-14):  {improvement_baseline:.1f}% improvement")
print(f"   vs LightGBM:          {improvement_lgbm:.1f}% {'improvement' if improvement_lgbm > 0 else 'worse'}")